## DeGirum `nnexpress` Compiler 
In this notebook, we illustrate the main compiler API that compiles a __tflite__ file to an __n2x__ file. The compiler API accepts a path to a tflite file and the device option. Currently, a tflite file can be compiled for CPU and ORCA (DeGirum's AI HW accelerator). To compile for CPU, specify ` device='SW' ` and to compile for ORCA, specify `device='HW'`

After the model is compiled, it can be saved as a file with extension __n2x__ and then run on HW and SW and the results can be compared to make sure that the model has been compiled correctly. The model's performance in terms of frames per second (FPS) can also be estimated. `nnexpress_utilities` contains helper functions to achieve these goals. 



In [ ]:
import nnexpress.compiler as n2xc
from nnexpress_utils import run_n2x, compare_n2x_sw_hw, estimate_fps

In [ ]:
import os

tflite_directory_str='/home/degirum/DeGirumModels/TFLiteModels/'
n2xsw_directory_str='/home/degirum/DeGirumModels/N2XSWModels/'
n2xhw_directory_str='/home/degirum/DeGirumModels/N2XHWModels/'

tflite_directory = os.fsencode(tflite_directory_str)
n2xsw_directory = os.fsencode(n2xsw_directory_str)
n2xhw_directory = os.fsencode(n2xhw_directory_str)

file_number=0    
for file in os.listdir(tflite_directory):
    filename = os.fsdecode(file)
    if filename.endswith(".tflite"): 
        print(file_number)
        file_number+=1
        tflite_path=os.path.join(tflite_directory, file)
        n2x_sw_path = os.path.join(n2xsw_directory, os.fsencode(filename.split('.')[0]+"_sw.n2x"))
        n2x_hw_path = os.path.join(n2xhw_directory, os.fsencode(filename.split('.')[0]+"_hw.n2x"))
        print('Compiling ',filename)
        sw_model=n2xc.Compiler(tflite_path, device='SW')
        hw_model=n2xc.Compiler(tflite_path, device='HW')
        sw_model.save(n2x_sw_path)
        hw_model.save(n2x_hw_path)
        print('Running SW n2x file:',run_n2x(n2x_sw_path))
        print('Running HW n2x file:',run_n2x(n2x_hw_path))
        match=compare_n2x_sw_hw(n2x_sw_path,n2x_hw_path)
        print('Comparing SW vs HW\n')
        if match:
            print('N2X SW and HW match')
        else:
            print('N2X SW and HW do not match')
        fps=estimate_fps(n2x_hw_path)
        print('FPS=',fps)
        print('**********************************************\n')

